In [1]:
'''
在Sep29版的基础上做出了如下修改：
把数据来源改为预备体变化。
'''

'\n在Sep29版的基础上做出了如下修改：\n把数据来源改为预备体变化。\n'

In [2]:
import os

# 获取当前工作目录
current_dir = os.getcwd()
print("当前工作目录：", current_dir)

# 修改当前工作目录，以后输出文件只需要写文件名
new_dir = "D:/李娅宁/9月下旬项目重开"
os.chdir(new_dir)
print("修改后的工作目录：", os.getcwd())

# 设置输入输出文件路径
obj_input_dir = "肩台外侧点-0715/"
marks1_input_dir = "肩台外侧点-0715/"
marks2_input_dir = "多分类0922/"
output_dir = "多分类预处理0930/"

当前工作目录： C:\Users\HP
修改后的工作目录： D:\李娅宁\9月下旬项目重开


In [3]:
import chardet
import numpy as np

def detect_encoding(file_path):
    '''检测文件的encoding方式'''
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        return result['encoding']
        
def load_obj_file(file_path, encoding):
    '''读取obj文件中的点云坐标，输出是list'''
    vertices = []
    faces = []
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            for line in file:
                if line.startswith('v '):
                    parts = line.strip().split()
                    vertex = [float(parts[1]), float(parts[2]), float(parts[3])]
                    vertices.append(vertex)
                elif line.startswith('f '):
                    parts = line.strip().split()
                    face = [int(p.split('/')[0]) - 1 for p in parts[1:]]
                    faces.append(face)
    except FileNotFoundError:
        print(f"文件未找到: {file_path}")
    except Exception as e:
        print(f"发生错误: {e}")
    return vertices, faces

def insert_midpoint_points(vertices, faces, minimum_vertices_number=3000):
    if len(vertices) >= minimum_vertices_number:
        print('原始点云的点数足够多')
        return vertices, faces

    else:
        temp = 0
        while len(vertices) < minimum_vertices_number:
            edge_to_midpoint = {}
            new_points = []
            new_faces = []
            vertex_offset = len(vertices)
    
            for face in faces:
                # Compute midpoints for each edge
                midpoints = []
                for i in range(len(face)):
                    edge = tuple(sorted((face[i], face[(i + 1) % len(face)])))
                    if edge not in edge_to_midpoint:
                        midpoint = np.mean([vertices[edge[0]], vertices[edge[1]]], axis=0)
                        edge_to_midpoint[edge] = vertex_offset + len(new_points)
                        new_points.append(midpoint.tolist())
                    midpoints.append(edge_to_midpoint[edge])
                
                # Original vertices
                v0, v1, v2 = face
                # Midpoints
                m0, m1, m2 = midpoints

                # Create four new faces
                new_faces.append([v0, m0, m2])
                new_faces.append([v1, m1, m0])
                new_faces.append([v2, m2, m1])
                new_faces.append([m0, m1, m2])
    
            vertices.extend(new_points)
            faces = new_faces
            temp += 1

        print(f'使用了 {temp} 轮插值来让点云点数满足要求')
        
    return vertices, faces

def center_vertices(vertices):
    '''将点云坐标系原点移动至其包围盒中心，输出是array'''
    vertices_array = np.array(vertices)
    min_coords = vertices_array.min(axis=0)
    max_coords = vertices_array.max(axis=0)
    center = (min_coords + max_coords) / 2
    centered_vertices = vertices_array - center
    return centered_vertices


In [4]:
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt

def compute_bounding_box(points):
    """计算点云的包围盒"""
    min_point = np.min(points, axis=0)
    max_point = np.max(points, axis=0)
    bbox_size = max_point - min_point
    bbox_center = (min_point + max_point) / 2.0
    return bbox_size, bbox_center

def extract_top_subcloud(points, bbox_size, bbox_center, height=4.0):
    """提取y坐标在ymax到ymax-3范围内的子点云"""
    min_y = bbox_center[1] + bbox_size[1] / 2.0 - height
    top_subcloud = points[points[:, 1] >= min_y]
    return top_subcloud

def find_best_rotation(points, axis1, axis2, axis3, half_angle_range=int(90), angle_step=1):
    """绕axis1旋转，使得axis2（优先）、axis3方向包围盒尺寸最小，输出旋转矩阵"""
    min_size2 = float('inf')
    min_size3 = float('inf')
    best_rotation_matrix = None

    for angle in np.arange(-half_angle_range, half_angle_range, angle_step):
        # 绕指定轴旋转
        rotation_matrix = R.from_euler(axis1, angle, degrees=True).as_matrix()
        rotated_points = points.dot(rotation_matrix.T)
        
        # 计算包围盒尺寸
        bbox_size, _ = compute_bounding_box(rotated_points)
        
        # 找出目标轴之外的最小尺寸
        if axis2 == 'x':
            size2 = bbox_size[0]  # x方向宽度
        elif axis2 == 'y':
            size2 = bbox_size[1]  # z方向宽度
        elif axis2 == 'z':
            size2 = bbox_size[2]  # z方向宽度

        # 找出目标轴之外的最小尺寸
        if axis3 == 'x':
            size3 = bbox_size[0]  # x方向宽度
        elif axis3 == 'y':
            size3 = bbox_size[1]  # z方向宽度
        elif axis3 == 'z':
            size3 = bbox_size[2]  # z方向宽度

        # 如果当前尺寸比之前的最小尺寸还小，则更新
        if size2 < min_size2:
            min_size2 = size2
            min_size3 = size3
            best_rotation_matrix = rotation_matrix
        elif size2 == min_size2:
            if size3 <min_size3:
                min_size3 = size3
                best_rotation_matrix = rotation_matrix

    return best_rotation_matrix

def flatten_and_analyze_curve(points, x_threshold=0.3, y_offset=1.5):
    """在新y-z平面上分析曲线的曲率，并确定x、z方向。
    
    参数:
    - points: 点云数据，形状为 (n, 3)
    - x_threshold: 用于滤波的x坐标阈值
    - y_offset: 确定用于分裂曲线的横线 y=ymax-y_offset 的偏移量
    """
    # 提取平面内的点
    flat_points = points[np.abs(points[:, 0]) <= x_threshold]

    # 提取 y 和 z 坐标
    y_coords = flat_points[:, 1]
    z_coords = flat_points[:, 2]

    # 对 y_coords 和 z_coords 进行排序以确保 z_coords 严格递增
    sorted_indices = np.argsort(z_coords)
    z_coords = z_coords[sorted_indices]
    y_coords = y_coords[sorted_indices]

    # 确定 ymax - y_offset 这条横线
    ymax = np.max(y_coords)
    y_threshold = ymax - y_offset

    # 找到 y_coords 刚刚超过 y_threshold 的最小和最大索引
    above_threshold_indices = np.where(y_coords > y_threshold)[0]
    if len(above_threshold_indices) == 0:
        raise ValueError("没有点的y坐标超过指定的y_threshold")

    min_index = above_threshold_indices[0]
    max_index = above_threshold_indices[-1]

    # 使用这些索引作为左侧和右侧的分界点
    left_indices = np.where((y_coords <= y_threshold) & (z_coords <= z_coords[min_index]))[0]
    right_indices = np.where((y_coords <= y_threshold) & (z_coords >= z_coords[max_index]))[0]

    left_index = left_indices[np.argmin(np.abs(y_coords[left_indices] - y_threshold))]
    right_index = right_indices[np.argmin(np.abs(y_coords[right_indices] - y_threshold))]

    y2_left, z2_left = y_coords[left_index], z_coords[left_index]
    y2_right, z2_right = y_coords[right_index], z_coords[right_index]

    # 找到曲线的第一个点和最后一个点
    y1, z1 = y_coords[0], z_coords[0]
    y3, z3 = y_coords[-1], z_coords[-1]

    # 计算线段 (z1, y1)-(z2_left, y2_left) 和 (z2_right, y2_right)-(z3, y3) 的斜率
    slope_left = (y2_left - y1) / (z2_left - z1)
    slope_right = (y3 - y2_right) / (z3 - z2_right)

    # 计算两侧线段下方的点数占比
    left_side_points = flat_points[flat_points[:, 2] < z2_left]
    right_side_points = flat_points[flat_points[:, 2] >= z2_right]

    left_under_line = left_side_points[:, 1] < (slope_left * (left_side_points[:, 2] - z1) + y1)
    right_under_line = right_side_points[:, 1] < (slope_right * (right_side_points[:, 2] - z2_right) + y2_right)

    left_under_ratio = np.sum(left_under_line) / len(left_side_points)
    right_under_ratio = np.sum(right_under_line) / len(right_side_points)

    # 确定 z 方向：下凹一侧为 z 正方向
    z_direction = -1 if left_under_ratio > right_under_ratio else 1

    # 复制相同的旋转给 z 方向以保持手性
    x_direction = z_direction

    return x_direction, z_direction

def align_point_cloud(points):
    """处理并旋转点云"""  
    # 计算原始点云的包围盒
    bbox_size, bbox_center = compute_bounding_box(points)

    # 提取顶端子点云1：较短，用于找到最扁取向
    top_subcloud1 = extract_top_subcloud(points, bbox_size, bbox_center, height=3.0)

    # 找到使得点云最扁的旋转角度
    best_rotation_matrix = find_best_rotation(top_subcloud1, 'y', 'z', 'x')

    # 提取顶端子点云2：较长，用于对比唇舌侧的凸性
    top_subcloud2 = extract_top_subcloud(points, bbox_size, bbox_center, height=5.0)
    
    # 将子点云旋转到最佳角度
    rotated_subcloud = top_subcloud2.dot(best_rotation_matrix.T)
    
    # 在新y-z平面上分析曲线并确定z轴方向
    x_direction, z_direction = flatten_and_analyze_curve(rotated_subcloud)

    # 确定最终旋转矩阵
    new_x_axis = best_rotation_matrix[:, 0] * x_direction
    new_y_axis = best_rotation_matrix[:, 1]
    new_z_axis = best_rotation_matrix[:, 2] * z_direction

    best_rotation_matrix_y = np.column_stack((new_x_axis, new_y_axis, new_z_axis))

    # 旋转整个原始点云到新的坐标系
    aligned_points = points.dot(best_rotation_matrix_y.T)

    # 在x轴上进行进一步优化旋转
    best_rotation_matrix_x = find_best_rotation(aligned_points, 'x', 'z', 'y', half_angle_range=int(30))
    aligned_points = aligned_points.dot(best_rotation_matrix_x.T)

    # 在z轴上进行进一步优化旋转
    best_rotation_matrix_z = find_best_rotation(aligned_points, 'z', 'x', 'y', half_angle_range=int(30))
    aligned_points = aligned_points.dot(best_rotation_matrix_z.T)

    # 最终的旋转矩阵
    final_rotation_matrix = best_rotation_matrix_y.T.dot(best_rotation_matrix_x.T).dot(best_rotation_matrix_z.T)

    return aligned_points, final_rotation_matrix


In [5]:
def special_posture_adjustment(points):
    '''由于marks2的csv文件是在早期版本的姿态调整的基础上生成的，
    要对marks2执行新姿态调整必须先计算旧姿态调整，
    这个函数用于计算旧姿态调整，最终总姿态调整将分两步执行。'''
    
    def sepcial_flatten_and_analyze_curve(points, z_threshold=0.3, y_offset=1.5):
        """在新x-y平面上分析曲线的曲率，并确定x、z方向。
        
        参数:
        - points: 点云数据，形状为 (n, 3)
        - z_threshold: 用于滤波的z坐标阈值
        - y_offset: 确定用于分裂曲线的横线 y=ymax-y_offset 的偏移量
        """
        # 提取平面内的点
        flat_points = points[np.abs(points[:, 2]) <= z_threshold]
    
        # 提取 x 和 y 坐标
        x_coords = flat_points[:, 0]
        y_coords = flat_points[:, 1]
    
        # 对 x_coords 和 y_coords 进行排序以确保 x_coords 严格递增
        sorted_indices = np.argsort(x_coords)
        x_coords = x_coords[sorted_indices]
        y_coords = y_coords[sorted_indices]
    
        # 确定 ymax - y_offset 这条横线
        ymax = np.max(y_coords)
        y_threshold = ymax - y_offset
    
        # 找到 y_coords 刚刚超过 y_threshold 的最小和最大索引
        above_threshold_indices = np.where(y_coords > y_threshold)[0]
        if len(above_threshold_indices) == 0:
            raise ValueError("没有点的y坐标超过指定的y_threshold")
    
        min_index = above_threshold_indices[0]
        max_index = above_threshold_indices[-1]
    
        # 使用这些索引作为左侧和右侧的分界点
        left_indices = np.where((y_coords <= y_threshold) & (x_coords <= x_coords[min_index]))[0]
        right_indices = np.where((y_coords <= y_threshold) & (x_coords >= x_coords[max_index]))[0]
    
        left_index = left_indices[np.argmin(np.abs(y_coords[left_indices] - y_threshold))]
        right_index = right_indices[np.argmin(np.abs(y_coords[right_indices] - y_threshold))]
    
        x2_left, y2_left = x_coords[left_index], y_coords[left_index]
        x2_right, y2_right = x_coords[right_index], y_coords[right_index]
    
        # 找到曲线的第一个点和最后一个点
        x1, y1 = x_coords[0], y_coords[0]
        x3, y3 = x_coords[-1], y_coords[-1]
    
        # 计算线段 (x1, y1)-(x2_left, y2_left) 和 (x2_right, y2_right)-(x3, y3) 的斜率
        slope_left = (y2_left - y1) / (x2_left - x1)
        slope_right = (y3 - y2_right) / (x3 - x2_right)
    
        # 计算两侧线段下方的点数占比
        left_side_points = flat_points[flat_points[:, 0] < x2_left]
        right_side_points = flat_points[flat_points[:, 0] >= x2_right]
    
        left_under_line = left_side_points[:, 1] < (slope_left * (left_side_points[:, 0] - x1) + y1)
        right_under_line = right_side_points[:, 1] < (slope_right * (right_side_points[:, 0] - x2_right) + y2_right)
    
        left_under_ratio = np.sum(left_under_line) / len(left_side_points)
        right_under_ratio = np.sum(right_under_line) / len(right_side_points)
    
        # 确定 x 方向：下凹一侧为 x 正方向
        x_direction = -1 if left_under_ratio > right_under_ratio else 1
    
        # 复制相同的旋转给 z 方向以保持手性
        z_direction = x_direction
    
        return x_direction, z_direction
    
    def sepcial_align_point_cloud(points):
        """处理并旋转点云"""  
        # 计算原始点云的包围盒
        bbox_size, bbox_center = compute_bounding_box(points)
    
        # 提取顶端子点云1：较短，用于找到最扁取向
        top_subcloud1 = extract_top_subcloud(points, bbox_size, bbox_center, height=3.0)
    
        # 找到使得点云最扁的旋转角度
        best_rotation_matrix = find_best_rotation(top_subcloud1, 'y', 'x', 'z')
    
        # 提取顶端子点云2：较长，用于对比唇舌侧的凸性
        top_subcloud2 = extract_top_subcloud(points, bbox_size, bbox_center, height=5.0)
        
        # 将子点云旋转到最佳角度
        rotated_subcloud = top_subcloud2.dot(best_rotation_matrix.T)
        
        # 在新x-y平面上分析曲线并确定x轴方向
        x_direction, z_direction = sepcial_flatten_and_analyze_curve(rotated_subcloud)
    
        # 确定最终旋转矩阵
        new_x_axis = best_rotation_matrix[:, 0] * x_direction
        new_y_axis = best_rotation_matrix[:, 1]
        new_z_axis = best_rotation_matrix[:, 2] * z_direction
    
        best_rotation_matrix_y = np.column_stack((new_x_axis, new_y_axis, new_z_axis))
    
        # 旋转整个原始点云到新的坐标系
        aligned_points = points.dot(best_rotation_matrix_y.T)
    
        # 在z轴上进行进一步优化旋转
        best_rotation_matrix_z = find_best_rotation(aligned_points, 'z', 'x', 'y', half_angle_range=int(30))
        aligned_points = aligned_points.dot(best_rotation_matrix_z.T)
    
        # 在x轴上进行进一步优化旋转
        best_rotation_matrix_x = find_best_rotation(aligned_points, 'x', 'z', 'y', half_angle_range=int(30))
        aligned_points = aligned_points.dot(best_rotation_matrix_x.T)
    
        # 最终的旋转矩阵
        final_rotation_matrix = best_rotation_matrix_y.T.dot(best_rotation_matrix_z.T).dot(best_rotation_matrix_x.T)
    
        return aligned_points, final_rotation_matrix

    special_aligned_points, special_rotation_matrix = sepcial_align_point_cloud(points)
    
    return special_aligned_points, special_rotation_matrix


In [6]:
# 创建一个两行一列的子图布局
def plot_cloud_before_and_after_pose_adjustment(points, aligned_points, save_path = None, y_rotation_angle=0):
    """绘制结果"""

    # 绘图
    fig = plt.figure(figsize=(14, 7))

    # 在第一个子图中显示原始点云
    ax1 = fig.add_subplot(121, projection='3d')
    ax1.scatter(points[:, 0], points[:, 1], points[:, 2], c='b', marker='.')
    # 旋转视角，将y轴竖直并绕y轴旋转
    ax1.view_init(elev=90, azim=y_rotation_angle)
    # 设置图例
    ax1.set_title('Original Point Cloud')
    ax1.set_xlabel('X axis')
    ax1.set_ylabel('Y axis')
    ax1.set_zlabel('Z axis')

    # 保持原始点云各轴比例一致
    max_range = np.array([points[:, 0].max() - points[:, 0].min(),
                          points[:, 1].max() - points[:, 1].min(),
                          points[:, 2].max() - points[:, 2].min()]).max() / 2.0

    mid_x = (points[:, 0].max() + points[:, 0].min()) * 0.5
    mid_y = (points[:, 1].max() + points[:, 1].min()) * 0.5
    mid_z = (points[:, 2].max() + points[:, 2].min()) * 0.5

    ax1.set_xlim(mid_x - max_range, mid_x + max_range)
    ax1.set_ylim(mid_y - max_range, mid_y + max_range)
    ax1.set_zlim(mid_z - max_range, mid_z + max_range)

    # 在第二个子图中显示对齐后的点云
    ax2 = fig.add_subplot(122, projection='3d')
    ax2.scatter(aligned_points[:, 0], aligned_points[:, 1], aligned_points[:, 2], c='r', marker='.')
    # 旋转视角，将y轴竖直并绕y轴旋转
    ax2.view_init(elev=90, azim=y_rotation_angle)
    # 设置图例
    ax2.set_title('Aligned Point Cloud')
    ax2.set_xlabel('X axis')
    ax2.set_ylabel('Y axis')
    ax2.set_zlabel('Z axis')

    # 设置对齐后的点云的坐标轴范围，使得每个轴的单位长度相同
    max_range_aligned = np.array([aligned_points[:, 0].max() - aligned_points[:, 0].min(),
                                  aligned_points[:, 1].max() - aligned_points[:, 1].min(),
                                  aligned_points[:, 2].max() - aligned_points[:, 2].min()]).max() / 2.0

    mid_x_aligned = (aligned_points[:, 0].max() + aligned_points[:, 0].min()) * 0.5
    mid_y_aligned = (aligned_points[:, 1].max() + aligned_points[:, 1].min()) * 0.5
    mid_z_aligned = (aligned_points[:, 2].max() + aligned_points[:, 2].min()) * 0.5

    ax2.set_xlim(mid_x_aligned - max_range_aligned, mid_x_aligned + max_range_aligned)
    ax2.set_ylim(mid_y_aligned - max_range_aligned, mid_y_aligned + max_range_aligned)
    ax2.set_zlim(mid_z_aligned - max_range_aligned, mid_z_aligned + max_range_aligned)

    # 保存图像
    if save_path:
        plt.savefig(save_path)
    else:
        plt.show()

    plt.close(fig)  # 关闭图像以释放内存


In [7]:
import networkx as nx
from scipy.interpolate import make_interp_spline

def load_mark_file(file_path, encoding):
    '''读取mark文件，对应marks1的数据'''
    marks = []
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            for line in file:
                parts = line.strip().split()
                if len(parts) == 3:
                    mark = [float(parts[0]), float(parts[1]), float(parts[2])]
                    marks.append(mark)
    except FileNotFoundError:
        print(f"文件未找到: {file_path}")
    except Exception as e:
        print(f"发生错误: {e}")
    return marks

def interpolate_spline(aligned_marks1):
    """
    对标记点进行样条插值，形成一个闭合曲线。点的顺序由极角排序确定。
    
    参数:
    aligned_marks1: 一个 Nx3 的数组，包含标记点的坐标。
    
    返回:
    spline_points: 插值后的点的坐标。
    """
    # 确保 aligned_marks1 是一个 numpy 数组
    aligned_marks1 = np.array(aligned_marks1)

    # 极角排序
    centered_aligned_marks1 = aligned_marks1 - np.mean(aligned_marks1)
    angles = np.arctan2(centered_aligned_marks1[:, 2], centered_aligned_marks1[:, 0])
    sorted_indices = np.argsort(angles)
    ordered_marks = aligned_marks1[sorted_indices]
    
    # 生成闭环
    closed_marks = np.vstack([ordered_marks, ordered_marks[0]])
    
    # 提取 x, y, z 坐标
    x = closed_marks[:, 0]
    y = closed_marks[:, 1]
    z = closed_marks[:, 2]
    
    # 生成参数 t
    t = np.linspace(0, 1, len(closed_marks))
    
    # 样条插值
    spline_x = make_interp_spline(t, x, k=3)
    spline_y = make_interp_spline(t, y, k=3)
    spline_z = make_interp_spline(t, z, k=3)
    
    # 生成更多的点
    t_new = np.linspace(0, 1, 101)  # 最后一个点会与起点重合，因此实际上是100个点
    spline_x_new = spline_x(t_new)
    spline_y_new = spline_y(t_new)
    spline_z_new = spline_z(t_new)
    
    # 合并第一个点以闭合曲线
    spline_x_new = np.concatenate([spline_x_new, [spline_x_new[0]]])
    spline_y_new = np.concatenate([spline_y_new, [spline_y_new[0]]])
    spline_z_new = np.concatenate([spline_z_new, [spline_z_new[0]]])
    
    spline_points = np.vstack([spline_x_new, spline_y_new, spline_z_new]).T
        
    return spline_points


In [8]:
def read_degree_file_to_graph(file_path):
    """
    从 .degree 文件中读取边并创建图形拓扑。
    
    输入:
    - file_path: .degree 文件的路径，文件中的每行包含两个数字，表示图的一条边
    
    输出:
    - G: 一个 networkx 图对象，包含了文件中定义的所有边
    """
    G = nx.Graph()  # 创建一个空的无向图
    with open(file_path, 'r') as file:
        for line in file:
            # 去掉行末的换行符并分割成两个索引
            node1, node2 = map(int, line.strip().split())
            G.add_edge(node1, node2)  # 添加边到图中
    return G

def find_intersections_and_paths(G2):
    """
    找到 G2 中的节点，并确定节点间的路径。
    如果一条路径能够分解为几条更短的路径（即路径上通过其他节点），那么只储存更短的路径。
    
    参数:
    G2: networkx 图，包含图的边信息。
    
    返回:
    filtered_paths: 路径列表。
    intersections: 节点的列表。
    """
    # 识别 G2 中度大于2的点作为节点
    intersections = [node for node, degree in dict(G2.degree()).items() if degree > 2]

    # 初始化原子路径
    filtered_paths = []

    # 遍历所有节点对
    for i in range(len(intersections)):
        for j in range(i + 1, len(intersections)):
            node1 = intersections[i]
            node2 = intersections[j]

            # 检查node1和node2之间是否存在不经过其他节点的路径
            # 创建一个子图，只保留node1和node2之间的路径
            subgraph = G2.copy()
            # 移除所有度大于等于3的节点（“节点”）并保留边
            for node in intersections:
                if node != node1 and node != node2:
                    subgraph.remove_node(node)

            # 检查node1和node2之间是否存在路径
            if nx.has_path(subgraph, node1, node2):
                # 获取路径并添加到路径列表中
                path = nx.shortest_path(subgraph, source=node1, target=node2)
                filtered_paths.append(path)
    
    return filtered_paths, intersections

def interpolate_spline_open_path(path, mark_points):
    """
    对给定的路径进行插值。如果路径长度大于3，使用三阶样条插值；否则使用线性插值。
    
    参数:
    path: 要插值的路径，节点的列表。
    mark_points: 对应 G2 中各点坐标。
    
    返回:
    spline_points: 插值后的点的坐标。
    """
    # 提取路径上的点
    ordered_marks = np.array([mark_points[i] for i in path])
    
    # 提取 x, y, z 坐标
    x = ordered_marks[:, 0]
    y = ordered_marks[:, 1]
    z = ordered_marks[:, 2]
    
    if len(path) > 3:
        # 对于长度大于3的路径，进行三阶样条插值
        t = np.linspace(0, 1, len(path))
        spline_x = make_interp_spline(t, x, k=3)
        spline_y = make_interp_spline(t, y, k=3)
        spline_z = make_interp_spline(t, z, k=3)
        # 插值点数 100
        t_new = np.linspace(0, 1, 100)
        spline_x_new = spline_x(t_new)
        spline_y_new = spline_y(t_new)
        spline_z_new = spline_z(t_new)
        
    else:
        # 对于长度小于等于3的路径，进行线性插值，插值点数 10
        t_new = np.linspace(0, 1, 10)
        spline_x_new = np.interp(t_new, [0, 1], [x[0], x[-1]])
        spline_y_new = np.interp(t_new, [0, 1], [y[0], y[-1]])
        spline_z_new = np.interp(t_new, [0, 1], [z[0], z[-1]])
    
    return np.vstack([spline_x_new, spline_y_new, spline_z_new]).T


In [9]:
def plot_contour(spline_points1, aligned_marks1, paths, aligned_marks2, save_path = None):
    '''绘制插值轮廓线'''
    fig = plt.figure(figsize=(14, 7))

    # 在第一个子图中显示G1
    ax1 = fig.add_subplot(121, projection='3d')
    ax1.plot(aligned_marks1[:,0], aligned_marks1[:,1], aligned_marks1[:,2], 'o', label='Original Points')
    ax1.plot(spline_points1[:,0], spline_points1[:,1], spline_points1[:,2], '-', label='Spline Curve')
    ax1.set_title('Aligned Marks1 and Spline Curve')
    ax1.set_xlabel('X axis')
    ax1.set_ylabel('Y axis')
    ax1.set_zlabel('Z axis')

    # 在第二个子图中显示G2
    ax2 = fig.add_subplot(122, projection='3d')
    # 绘制marks点
    ax2.scatter(aligned_marks2[:, 0], aligned_marks2[:, 1], aligned_marks2[:, 2], c='r', marker='o')
    # 绘制每条路径
    for path in paths:
        ax2.plot(path[:, 0], path[:, 1], path[:, 2], '-')
    ax2.set_title('Aligned Marks2 and Spline Curve')
    ax2.set_xlabel('X axis')
    ax2.set_ylabel('Y axis')
    ax2.set_zlabel('Z axis')
    
    # 保存图像
    if save_path:
        plt.savefig(save_path)
    else:
        plt.show()

    plt.close(fig)  # 关闭图像以释放内存


In [10]:
from scipy.spatial import cKDTree
from sklearn.neighbors import KDTree

def find_nearby_points(vertices, curve_points, tolerance):
    """
    在曲线附近找到宽度为 tolerance 的点。
    
    参数:
    vertices: 点云数据。
    curve_points: 样条插值生成的曲线点。
    tolerance: 宽度容忍度。
    
    返回:
    nearby_points_indices: 在曲线附近的点的索引。
    """
    # 使用 KDTree 查找邻域
    tree = cKDTree(vertices)
    nearby_points_indices = set()
    
    for point in curve_points:
        indices = tree.query_ball_point(point, tolerance)
        nearby_points_indices.update(indices)
    
    return nearby_points_indices

def generate_graph(vertices, faces):
    """
    生成点云的拓扑图。
    """
    G = nx.Graph()
    for i, vertex in enumerate(vertices):
        G.add_node(i, pos=vertex)
    for face in faces:
        for i in range(len(face)):
            G.add_edge(face[i], face[(i + 1) % len(face)])
    return G

def segment_point_cloud(vertices, faces, spline_points):
    """
    执行点云分割。通过逐步增加tolerance的方式移除spline_points附近的点，直到图被分割为8个大块。
    """
    G = generate_graph(vertices, faces)  # 生成拓扑图

    tolerance = 0.1
    while tolerance <= 0.25:
        nearby_points = find_nearby_points(vertices, spline_points, tolerance)
        G.remove_nodes_from(nearby_points)
        components = list(nx.connected_components(G))
        large_components = [comp for comp in components if len(comp) > 20]  # 规定点数过阈值的组件才能被视为真正的组件

        if len(large_components) == 8:
            print(f"分割成功，容差为：{tolerance}")
            
            # y 均值最大和最小的分别是切端和牙根
            sorted_components = sorted(large_components, key=lambda comp: np.mean([vertices[i][1] for i in comp]))
            part1 = sorted_components[0]  # 牙根
            part2 = sorted_components[-1]  # 切端

            # 剩下部分中 x 均值最大和最小的分别是近中和远中
            remaining_parts = sorted_components[1:-1]
            remaining_parts.sort(key=lambda comp: np.mean([vertices[i][0] for i in comp]))
            part3 = remaining_parts[-1]  # 近中
            part4 = remaining_parts[0]  # 远中

            # 剩下部分中 z 均值最大和最小的分别是舌轴面和唇面
            remaining_parts = remaining_parts[1:-1]
            remaining_parts.sort(key=lambda comp: np.mean([vertices[i][2] for i in comp]))
            part5 = remaining_parts[-1]  # 舌轴面
            part6 = remaining_parts[0]  # 唇面

            # 剩下部分中 y 均值较大的是舌面凹，较小的是肩台
            remaining_parts = remaining_parts[1:-1]
            remaining_parts.sort(key=lambda comp: np.mean([vertices[i][1] for i in comp]))
            part7 = remaining_parts[0]  # 肩台
            part8 = remaining_parts[1]  # 舌面窝

            return [part1, part2, part3, part4, part5, part6, part7, part8], nearby_points

        tolerance += 0.005

    print(f"分割失败，组件数：{len(large_components)}")
    return [], nearby_points

def perform_voting(vertices, parts):
    """
    为点云数据中的每个点分配标签，确保每个 part 的标签与其索引一致，同时使用投票机制对未标记的点进行标签分配。
    
    参数:
    vertices: 点云数据，形状为 (n, 3)，每行表示一个点的 (x, y, z) 坐标。
    parts: 列表，包含每个分区的点的索引列表。
    
    返回:
    labels: 点的标签，形状为 (n,)，每个标签是一个整数。
    """
    # 创建 KDTree 以高效地查找邻域
    tree = KDTree(vertices)
    
    # 创建标签数组
    labels = np.zeros(len(vertices), dtype=int)
    
    # 为每个 part 分配标签
    for i, part in enumerate(parts):
        for idx in part:
            labels[idx] = i + 1  # 标签从1开始
    
    # 临时标签数组用于存储投票结果
    temp_labels = np.zeros(len(vertices), dtype=int)
    
    # 设置邻域半径的初始值和最大值
    r_min = 0.2
    r_max = 1
    r_increment = 0.2
    
    # 对每个未标记的点进行投票
    for i, vertex in enumerate(vertices):
        if labels[i] > 0:
            continue  # 跳过已经标记的点
        
        # 初始化邻域半径
        r = r_min
        found_valid_neighbor = False
        
        while r <= r_max:
            # 查找邻居
            neighbors, distances = tree.query_radius([vertex], r=r, return_distance=True)
            neighbors = neighbors[0]  # 提取索引数组
            distances = distances[0]  # 提取距离数组
            
            # 只考虑已标记的邻居进行投票
            valid_neighbors = neighbors[labels[neighbors] > 0]
            valid_distances = distances[labels[neighbors] > 0]
            
            if len(valid_neighbors) > 2:  # 确保有足够的有效邻居
                found_valid_neighbor = True
                break
            r += r_increment
        
        if not found_valid_neighbor:
            continue

        # 如果找到有效邻居，进行标签投票
        neighbor_labels = labels[valid_neighbors]
        unique_labels, counts = np.unique(neighbor_labels, return_counts=True)
        
        if len(unique_labels) > 0:
            # 获取最高计数的标签
            max_count = np.max(counts)
            candidates = unique_labels[counts == max_count]
            
            if len(candidates) > 1:  # 如果有多个标签计数相同，选择平均距离最小的标签
                avg_distances = np.array([np.mean(valid_distances[neighbor_labels == label]) for label in candidates])
                best_label = candidates[np.argmin(avg_distances)]
            else:
                best_label = candidates[0]
            
            temp_labels[i] = best_label
    
    # 更新最终标签
    labels[temp_labels > 0] = temp_labels[temp_labels > 0]
    
    return labels

def save_labeled_point_cloud(vertices, labels, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for i, vertex in enumerate(vertices):
            if labels[i] > 0:  # 仅保存已标记的点
                file.write(f"{vertex[0]} {vertex[1]} {vertex[2]} {labels[i]}\n")


In [11]:
def plot_labeled_point_cloud(vertices, labels, spline_points1, interpolated_paths, save_path = None):
    """
    绘制带有不同颜色标签的点云图。

    参数:
    vertices: 点云数据，形状为 (n, 3)，每行表示一个点的 (x, y, z) 坐标。
    labels: 点的标签，形状为 (n,)，每个标签是一个整数。
    colors_list: 每个标签的颜色列表，形状为 (m,)。
    labels_list: 每个标签的图例标签列表，形状为 (m,)。
    """
    # 创建 3D 图形
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # 获取唯一的标签
    unique_labels = np.unique(labels)
    
    # 设置颜色映射
    colors_list = ['gray', 'pink', 'gray', 'gray', 'green', 'blue', 'pink', 'blue']
    
    # 设置图例标签
    labels_list = ['Ya Gen', 'Qie Duan', 'Jin Zhong', 'Yuan Zhong', 'She Zhou Mian', 'Chun Mian', 'Jian Tai', 'She Mian Wo']

    # 设置透明度列表
    alpha_list = [1.0, 1.0, 0.5, 0.5, 1.0, 0.5, 1.0, 0.5]

    # 设置点的大小
    point_size = 0.1

    # 绘制每个标签的点
    for label in unique_labels:
        if label == 0:
            continue  # 跳过未标记的点

        # 获取当前标签的点
        indices = np.where(labels == label)[0]
        points = vertices[indices]

        # 使用列表中的信息
        color = colors_list[int(label) - 1] if colors_list else 'gray'
        get_label = labels_list[int(label)-1] if labels_list else 'unknown'
        alpha = alpha_list[int(label) - 1] if alpha_list else 0.5
        
        # 绘制点
        ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=color, label=get_label, alpha=alpha, s=point_size)

    # 绘制marks1线
    ax.plot(spline_points1[:, 0], spline_points1[:, 1], spline_points1[:, 2], '-', color='red')
    
    # 绘制marks2线
    for path in interpolated_paths:
        ax.plot(path[:, 0], path[:, 1], path[:, 2], '-', color='red')

    # 设置图例和标签
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.legend(loc='center left', bbox_to_anchor=(0.85, 0.55), title='Parts')
    plt.title('Labeled Point Cloud')

    # 保持原始点云各轴比例一致
    points = np.asarray(vertices)
    max_range = np.array([points[:, 0].max() - points[:, 0].min(),
                          points[:, 1].max() - points[:, 1].min(),
                          points[:, 2].max() - points[:, 2].min()]).max() / 2.0

    mid_x = (points[:, 0].max() + points[:, 0].min()) * 0.5
    mid_y = (points[:, 1].max() + points[:, 1].min()) * 0.5
    mid_z = (points[:, 2].max() + points[:, 2].min()) * 0.5

    ax.set_xlim(mid_x - max_range, mid_x + max_range)
    ax.set_ylim(mid_y - max_range, mid_y + max_range)
    ax.set_zlim(mid_z - max_range, mid_z + max_range)

    # 设置视角
    ax.view_init(elev=-30, azim=60)
    
    # 保存图像
    if save_path:
        plt.savefig(save_path)
    else:
        plt.show()

    plt.close(fig)  # 关闭图像以释放内存


In [12]:
def batch_process_obj_files(obj_input_dir, marks1_input_dir, marks2_input_dir, output_dir):
    for root, dirs, files in os.walk(obj_input_dir):
        for filename in files:
            if filename.endswith(".obj"):
                # 设置地址
                obj_file_path = os.path.join(root, filename)
                mark_file_path1 = os.path.join(root, filename + '.mark')
                mark_file_path21 = os.path.join(marks2_input_dir + 'mark/', filename + '.mark')
                mark_file_path22 = os.path.join(marks2_input_dir + 'degree/', filename + '.degree')
                txt_save_path = os.path.join(output_dir, "预处理结果文档/" + filename[:-4] + '.txt')
                image_save_path1 = os.path.join(output_dir, "姿态调整图像/" + filename[:-4] + '_aligned.png')
                image_save_path2 = os.path.join(output_dir, "轮廓线图像/" + filename[:-4] + '_contour.png')
                image_save_path3 = os.path.join(output_dir, "预处理结果图像/" + filename[:-4] + '_processed.png')

                # 读取点云坐标并中心化
                obj_encoding = detect_encoding(obj_file_path)
                obj_vertices, obj_faces = load_obj_file(obj_file_path, obj_encoding)
                checked_vertices, faces = insert_midpoint_points(obj_vertices, obj_faces, 5000)  # 对点数小于5000的点云做插值加密
                centered_vertices = center_vertices(checked_vertices)
    
                # 旧姿态调整
                special_aligned_points, special_rotation_matrix = special_posture_adjustment(centered_vertices)
                # 新姿态调整
                aligned_points, final_rotation_matrix = align_point_cloud(special_aligned_points)
    
                # 对比姿态调整前后的点云
                plot_cloud_before_and_after_pose_adjustment(centered_vertices, aligned_points, image_save_path1)
    
                # 读取marks1并进行插值
                mark_encoding1 = detect_encoding(mark_file_path1)
                marks1 = load_mark_file(mark_file_path1, mark_encoding1)
                aligned_marks1 = np.asarray(marks1).dot(special_rotation_matrix).dot(final_rotation_matrix)
                spline_points1 = interpolate_spline(aligned_marks1)
    
                # 读取marks2
                mark_encoding2 = detect_encoding(mark_file_path21)
                marks2 = load_mark_file(mark_file_path21, mark_encoding2)
                aligned_marks2 = np.asarray(marks2).dot(special_rotation_matrix).dot(final_rotation_matrix)
                
                G2 = read_degree_file_to_graph(mark_file_path22)
                paths, intersections = find_intersections_and_paths(G2)           
    
                # 对G2的所有路径进行插值
                interpolated_paths = []
                for path in paths:
                    interpolated_paths.append(interpolate_spline_open_path(path, aligned_marks2))
    
                # G2插值线上的所有点
                spline_points2 = np.unique(np.vstack(interpolated_paths), axis=0)
    
                # 合并G1和G2的所有插值点为一个数据,即完整轮廓线
                spline_points = np.unique(np.vstack([spline_points1, spline_points2]), axis=0)
    
                # 绘制轮廓线
                plot_contour(spline_points1, aligned_marks1, interpolated_paths, aligned_marks2, image_save_path2)
    
                # 执行点云分割和标签分配
                parts, points_to_remove = segment_point_cloud(aligned_points, faces, spline_points)
                if parts == []:
                    print(filename[:-4], "分割失败")
                labels = perform_voting(aligned_points, parts)
    
                # 绘制分割后的图像
                plot_labeled_point_cloud(aligned_points, labels, spline_points1, interpolated_paths, image_save_path3)
                
                # 保存结果
                save_labeled_point_cloud(aligned_points, labels, txt_save_path)


In [13]:
'''整'''
batch_process_obj_files(obj_input_dir, marks1_input_dir, marks2_input_dir, output_dir)

使用了 1 轮插值来让点云点数满足要求
分割成功，容差为：0.1950000000000001
使用了 1 轮插值来让点云点数满足要求


FileNotFoundError: [Errno 2] No such file or directory: '多分类0922/mark/1_10.obj.mark'

In [ ]:
'''
结果：我没有350颗的目录，只能测试，无法正式运行。
'''